## Import necessary packages

In [1]:
!pip install rouge-score
!pip install simpletransformers
import json
from datetime import datetime
from statistics import mean
import pandas as pd
from simpletransformers.t5 import T5Model

     |████████████████████████████████| 221 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 264 kB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 243 kB 57.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 46.5 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 3.1 MB/s  eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=1d0e0515a16ebefd600fa7d48cbddd068cb37f235e237f1356ec62d18ff9eac2
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
import numpy as np
import pandas as pd
import random
import time
import unicodedata
import string
import re
import random
import os
import nltk
from scipy.special import softmax


import json
import pandas as pd
import numpy as np
import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
from rouge_score import rouge_scorer

seed = 44  
random.seed(seed)
np.random.seed(seed)

In [3]:
# Load train and test data
train = pd.read_csv("../input/yor-en/train_yoruba.csv")
test = pd.read_csv("../input/yor-en/test_yoruba.csv")

# EDA & PROCESSING

In [4]:
#Check for duplicates

train = train[~train[['Yoruba','English']].duplicated()]

Check if numeric is needed in test set

In [5]:
find_nos = [re.search('[0-9]+', i).group() for i in train['English'] if re.search('[0-9]+', i) != None]

In [6]:
find_nos[:10]

['22', '2', '1550', '18', '351', '39', '900', '1', '1', '50']

In [7]:
train.head()

,Unnamed: 0,ID,Yoruba,English
0,0,ID_AAJEQLCz,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù l...,A Disaster Relief Committee was formed to orga...
1,1,ID_AASNedba,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákù...",Brother Solovyev was arrested on the evening o...
2,2,ID_AAeQrhMq,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,3,ID_AAxlMgPP,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo...","With Egypt, Morocco and Tunisia out of the Wor..."
4,4,ID_ABKuMKSx,Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney Gen...,"The Attorney General of the Federation, Justic..."


In [8]:
print("Training Set Shape :", train.shape)
print("Test Set Shape :", test.shape)

Training Set Shape : (9990, 4)
Test Set Shape : (6816, 3)


In [9]:
train['Eng_len'] = [len(i.split())for i in train['English']]

train['Yor_len'] = [len(i.split())for i in train ['Yoruba']]

In [10]:
test['Yor_len'] = [len(i.split())for i in test['Yoruba']]

In [11]:
train['Yor_len'].max(), train['Eng_len'].max()

(219, 159)

In [12]:
test['Yor_len'].max()

129

In [13]:
np.quantile(train['Eng_len'], 0.999), np.quantile(train['Yor_len'], 0.999), 

(84.01100000000042, 104.08800000000338)

In [14]:
np.quantile(test['Yor_len'], 0.999), 

(83.0,)

## 128 Sentence length will do in this case. 99.99% are less than the number

In [15]:
train.reset_index(drop = True, inplace = True)

In [16]:
train_df = train[['Yoruba', 'English']]

In [17]:
train_df.head()

,Yoruba,English
0,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù l...,A Disaster Relief Committee was formed to orga...
1,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákù...",Brother Solovyev was arrested on the evening o...
2,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo...","With Egypt, Morocco and Tunisia out of the Wor..."
4,Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney Gen...,"The Attorney General of the Federation, Justic..."


## Loading & Splitting The Competition Data



In [18]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

In [19]:
train_df = train_df.rename(columns = {'Yoruba' : 'input_text', 'English' : 'target_text'})

In [20]:
# #splitting the training set in to training and validation sets
from sklearn.model_selection import train_test_split  
train_df, val =  train_test_split(train_df, test_size = 0.10  , random_state = 1)

In [21]:
len(val)

999

In [22]:
len(train_df)

8991

**Helper Functions for Competition Metric to check on Eval Set**

In [23]:
def create_metric(df, spec):

  scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=False)

  scores = scorer.score(df['target_text'], df[spec])

  return float(scores['rouge1'][2])

In [24]:
def metric_score(spec, val):

  val['scores'] = val.apply(create_metric, axis = 1, args = [spec])

  return val['scores'].mean()

## Translation Model MT5 Pretrained

**Pre-process Data for MT5 training**

In [25]:
train_df["prefix"] = ""
val["prefix"] = ""

In [26]:
train_df = train_df[["prefix", "input_text", "target_text"]]
val = val[["prefix", "input_text", "target_text"]]

**MT5 training params**

In [27]:
#Set Params

batch_size= 8
batch_steps = len(train_df)/batch_size
warmup = batch_steps/3

**Training-Validation Routine:**

In [28]:
args = ({"adam_epsilon": 1e-08, \
        "cosine_schedule_num_cycles": 0.5, 
        "do_lower_case": False, \
        "dynamic_quantize": False, \
        "early_stopping_delta": 0, \
        "early_stopping_metric": "eval_loss", \
        "early_stopping_metric_minimize": True, \
        "early_stopping_patience": 3, \
        "encoding": None, \
        "adafactor_eps": [1e-30, 0.001], \
        "adafactor_clip_threshold": 1.0, \
        "adafactor_decay_rate": -0.8, \
        "adafactor_beta1": None, \
        "adafactor_scale_parameter": False, \
        "adafactor_relative_step": False, 
        "adafactor_warmup_init": False, \
        "evaluate_during_training": True, \
        "evaluate_during_training_silent": False, \
        "evaluate_during_training_steps": batch_steps, \
        "evaluate_during_training_verbose": True, \
        "fp16": False, \
        "gradient_accumulation_steps": 1, \
        "learning_rate": 1e-5,
        "local_rank": -1, \
        "logging_steps": 50, \
        "manual_seed": 42, \
        "max_grad_norm": 1.0, \
        "max_seq_length": 128, \
        "num_train_epochs": 14, \
        "optimizer": "Adafactor", \
        "process_count": 254, \
        "polynomial_decay_schedule_lr_end": 1e-09, \
        "polynomial_decay_schedule_power": 1.0, \
        "quantized_model": False, \
        "reprocess_input_data": True, \
        "save_best_model": True, \
        "save_eval_checkpoints": False, \
        "save_optimizer_and_scheduler": False, \
        "save_steps": -1, \
        "scheduler": "constant_schedule_with_warmup", \
        "silent": False, \
        "skip_special_tokens": True, \
        "tokenizer_type": None, \
        "tokenizer_name": None, \
        "train_batch_size": batch_size, \
        "use_cached_eval_features": False, \
        "use_early_stopping": True,\
        "use_multiprocessing": False, \
        "use_multiprocessing_for_evaluation": False, \
        "warmup_ratio": 0.06, \
        "warmup_steps": warmup, \
        "dataset_class": None, \
        "do_sample": False, \
        "length_penalty": 2.0, \
        "max_length": 200, \
        "max_steps": -1, \
        "num_beams": 1, \
        "num_return_sequences": 1, \
        "preprocess_inputs": True, \
        "repetition_penalty": 1.0, \
        "special_tokens_list": [], \
        "top_k": None, \
        "top_p": None, \
        "use_multiprocessed_decoding": False, \
        "eval_batch_size" : batch_size, \
        "overwrite_output_dir" : True,
        "train_custom_parameters_only" : False, \
        "save_model_every_epoch": False,
        "custom_parameter_groups" :[{"params": ['decoder.block.11.layer.2.DenseReluDense.wi_0.weight',
                                                'decoder.block.11.layer.2.DenseReluDense.wi_1.weight',
                                                'decoder.block.11.layer.2.DenseReluDense.wo.weight',
                                                'decoder.block.11.layer.2.layer_norm.weight',
                                                'decoder.final_layer_norm.weight',
                                                'lm_head.weight']}]
})


#Load the trained model

full_model = T5Model("mt5", "Davlan/mt5_base_yor_eng_mt", args= args, use_cuda = True)

Downloading:   0%|          | 0.00/673 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/351 [00:00<?, ?B/s]

In [29]:
#Trainable MT5 Params- Training all layers of MT5 for fine-tuning, experimentation is possible.
full_model.get_named_parameters()

['shared.weight',
 'encoder.block.0.layer.0.SelfAttention.q.weight',
 'encoder.block.0.layer.0.SelfAttention.k.weight',
 'encoder.block.0.layer.0.SelfAttention.v.weight',
 'encoder.block.0.layer.0.SelfAttention.o.weight',
 'encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight',
 'encoder.block.0.layer.0.layer_norm.weight',
 'encoder.block.0.layer.1.DenseReluDense.wi_0.weight',
 'encoder.block.0.layer.1.DenseReluDense.wi_1.weight',
 'encoder.block.0.layer.1.DenseReluDense.wo.weight',
 'encoder.block.0.layer.1.layer_norm.weight',
 'encoder.block.1.layer.0.SelfAttention.q.weight',
 'encoder.block.1.layer.0.SelfAttention.k.weight',
 'encoder.block.1.layer.0.SelfAttention.v.weight',
 'encoder.block.1.layer.0.SelfAttention.o.weight',
 'encoder.block.1.layer.0.layer_norm.weight',
 'encoder.block.1.layer.1.DenseReluDense.wi_0.weight',
 'encoder.block.1.layer.1.DenseReluDense.wi_1.weight',
 'encoder.block.1.layer.1.DenseReluDense.wo.weight',
 'encoder.block.1.layer.1.layer_norm.

## Training MT5 Model on Train set, validating on hold out set

In [30]:
#%%time
full_model.train_model(train_df, eval_data = val)

  0%|          | 0/8991 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Using Adafactor for T5


Epoch:   0%|          | 0/14 [00:00<?, ?it/s]

Running Epoch 0 of 14:   0%|          | 0/1124 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

Running Epoch 13 of 14:   0%|          | 0/1124 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

(15736,
 {'global_step': [1124,
   2248,
   3372,
   4496,
   5620,
   6744,
   7868,
   8991,
   8992,
   10116,
   11240,
   12364,
   13488,
   14612,
   15736],
  'eval_loss': [2.0637564134597777,
   2.005174973487854,
   1.9674850168228148,
   1.9460718603134155,
   1.926972412109375,
   1.913552845954895,
   1.8997657928466798,
   1.8912156143188477,
   1.8911588563919068,
   1.8828787565231324,
   1.8798219289779663,
   1.8726026191711427,
   1.8721655731201172,
   1.868517746925354,
   1.8631209802627564],
  'train_loss': [1.789546012878418,
   1.9335169792175293,
   2.0942952632904053,
   2.4130265712738037,
   2.215636968612671,
   2.339406967163086,
   2.153193712234497,
   1.7890820503234863,
   1.4261651039123535,
   1.7732902765274048,
   1.233276605606079,
   2.1404659748077393,
   1.5579873323440552,
   1.4746031761169434,
   2.403543710708618]})

In [31]:
#Actual Validation Loss[Trained]

print(full_model.eval_model(val))

  0%|          | 0/999 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.8631209802627564}


#### Check overall score of trained model on validation set

In [32]:
# Prepare the data for validation(Train)
train_predict = [
        prefix + ": " + str(input_text)
        for prefix, input_text in zip(train_df["prefix"].tolist(), train_df["input_text"].tolist())
    ]

In [33]:
# Prepare the data for validation(val)
to_predict = [
        prefix + ": " + str(input_text)
        for prefix, input_text in zip(val["prefix"].tolist(), val["input_text"].tolist())
    ]

**Trained Model**

In [34]:
val['Label_MT5_trained'] = full_model.predict(to_predict)

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

**Score Metrics for Trained Model**

In [36]:
print("Rouge 1 Score on Validation for New trained model is {}".format(metric_score("Label_MT5_trained", val)))

Rouge 1 Score on Validation for New trained model is 0.46917067316673233


### TEST SET PREDICTION (MT5):

**Preprocess Test set for MT5**

In [37]:
test["prefix"] = ""

In [38]:
test = test.rename(columns = {'Yoruba' : 'input_text'})

In [39]:
# Prepare the data for testing
test_predict = [
        prefix + ": " + str(input_text)
        for prefix, input_text in zip(test["prefix"].tolist(), test["input_text"].tolist())
    ]

In [40]:
test['Label'] = full_model.predict(test_predict)

Generating outputs:   0%|          | 0/852 [00:00<?, ?it/s]

In [41]:
#Preprocess to remove semicolon

def preprocess_mt5(df):

  return re.sub(r'^:', "", df)

In [42]:
test['Label'] = test['Label'].apply(preprocess_mt5)

In [44]:
#Checking model translations

test[['input_text', 'Label']]

,input_text,Label
0,"Nínú ìpè kan lẹ́yìn ìgbà náà, wọ́n sọ fún aṣoj...","In a call later, the BlaBlaCar representative ..."
1,Nítorí kò sí nǹkan tí ọkùnrin ò lè ṣe láì náán...,For there is nothing a man can do without bein...
2,Bí i kó pariwo. Kí ó kígbe mọ́ ẹ?,"If he makes a noise, what will he say to you?"
3,"Tí ó ń lé e lọ sọ́nà etí odò Akókurà, tí ó bẹ̀...","which takes him to the coast of Akókurà, so th..."
4,Èṣúńiyì mọ̀ iṣẹ́ rẹ̀ dunjú. Màmá tirí bí ó ṣe ...,The Ṣuńiyi knows his work well. Mother told he...
...,...,...
6811,(Ẹ̀rín) Ètò náà ti wá ń ṣiṣẹ́ ní àwọn orílẹ̀-è...,(Clue) The program has been operating in nine ...
6812,Ẹtúra ká.,The leopard covers the ground.
6813,"Ní báyìí, ní àfikún Ebola àti awuyewuye ọ̀rọ̀ ...","Now, in addition to Ebola and the growing deba..."
6814,Ṣé mo yà ẹ́ lẹ́nu lọ́nàkọnà? Pẹ̀lẹ́ pẹ̀lẹ́.,Do I easily stumble you? Very gentle.
